# Getting started processing Kafka with Spark

The first thing we'll need to do is tell Spark where to find the Kafka driver before we set Spark up.  Currently, our notebook images are built against Spark 2.3.2.  If you're using this with a different version of Spark, be sure to change `SPARK_VERSION` in the cell below before executing it.

In [ ]:
import os
import findspark

SPARK_VERSION="2.3.2"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[2] --conf spark.jars.ivy=/opt/app-root/src/.ivy2 --packages org.apache.spark:spark-sql-kafka-0-10_2.11:%s pyspark-shell" % SPARK_VERSION
os.environ["SPARK_HOME"] = "/opt/app-root/lib/python3.6/site-packages/pyspark"
findspark.init()

Next up, we'll connect to Spark by establishing a `SparkSession`.

In [ ]:
import pyspark

from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local[2]") \
    .appName("Social Firehose") \
    .getOrCreate()

We're going to begin by loading the contents of a Kafka topic into a data frame.  Because Spark data frames are _lazy_, or recomputed when accessed, this data frame will always have the most recent collection of messages in it.

In [ ]:
df = spark \
  .read \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "odh-message-bus-kafka-bootstrap:9092") \
  .option("subscribe", "social-firehose") \
  .load()

We can see that this data frame always has the most recent collection of messages by running the `count()` action on it twice with a short delay in the middle.  Note how many messages are generated in ten seconds:

In [ ]:
import time
a = df.count()
time.sleep(10)
b = df.count()
(a, b)

We can inspect the first few messages, but they'll be in a pretty raw format.

In [ ]:
df.take(3)

Now we'll import some functions and types from the Spark library so we can do something more useful with our data set.

In [ ]:
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark.sql.functions import from_json
from pyspark.sql.functions import column
from pyspark.sql.types import StringType
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField

The first thing we'll do is extract the JSON payloads of the messages; we'll inspect the first ten as a sanity check.

In [ ]:
values = df.select(df["value"].cast(StringType()).alias("value"))
values.show(10)

The next thing we'll do is impose some structure on the messages by converting the serialized JSON objects into actual records:

1.  First, we'll declare a `StructType` for the structure of our messages (three strings, named `text`, `user_id`, and `update_id`),
2.  Next, we'll convert the JSON strings to structures using the `from_json` dataframe function, and
3.  Finally, we'll `SELECT` the fields of the object so we have something that looks like a flat database tuple.

In [ ]:
structure = StructType([StructField(fn, StringType(), True) for fn in "text user_id update_id".split()])
records = values.select(from_json(values["value"], structure).alias("json")) \
                .select(column("json.update_id"), column("json.user_id").alias("user_id"), column("json.text"))
records.show(10)

We can perform database-style aggregations on this data frame, like identifying the users responsible for the most status updates:

In [ ]:
user_counts = records.groupBy("user_id").count().orderBy("count", ascending=False)
user_counts.show()

If you run that query several times with a short delay in between, you may get different results since the data frame will reflect newly-arriving messages.  Try it out!

We can also count the number of users who have issued status updates (because of how we're generating the synthetic stream of updates, there is an upper bound on this number):

In [ ]:
records.select("user_id").distinct().count()

We can also identify the most prolix users.  You probably have some social media connections who take advantage of every extra bit of character limit; a query like this will show you who they are!

In [ ]:
from pyspark.sql.functions import length
user_loquacity = records.select(column("user_id"), length("text").alias("update_len")) \
  .groupBy("user_id") \
  .avg() \
  .orderBy("avg(update_len)", ascending=False)
user_loquacity.show()

We can also identify the most popular hashtags in users' updates.  We'll start by turning each update into an array of words.  Then we'll explode each array into multiple rows, so that each row has a separate, single element, i.e.

```
1, 2, "foo bar blah"
```

would become

```
1, 2, [foo, bar, blah]
```

which would become

```
1, 2, foo
1, 2, bar
1, 2, blah
```

We'll then filter for hashtags (keeping only words starting with `#`) so we can find the most popular!


In [ ]:
words = records.select(explode(split("text", " ")).alias("word"))
hashtags = words.filter(column("word").startswith("#"))

In [ ]:
words.show()

In [ ]:
hashtags.show()

In [ ]:
hashtag_counts = hashtags.groupBy("word").count().orderBy("count", ascending=False)
hashtag_counts.show()